# Frequent Itemsets

By Linus Östlund and Daniel Workinn

In this homework we implement one of the major families of techniques for characterizing data - discovering `Frequent Itemsets` and `Association Rules`.  

To discover `Frequent Itemsets` we have implemented the `A-Priori Algorithm` by following the guidelines in the course litterature. 

We then use the frequent itemsets to mine `Association Rules`. The association rules quantify the conditional probability of item `j` existing in the basket given itemset `I` (I -> J). Association rules can be used by e.g. a retailer to learn what items are commonly bought together. 

## Implementation

First, read the data from the file and create a list of baskets (also called *transactions*).

In [1]:
def import_baskets(path_to_data):
    with open(path_to_data) as f:
        lines = f.readlines()
        baskets = [set(map(int, line.strip().split(sep=' '))) for line in lines]
    return baskets

def get_unique_items(baskets):
    """
    Helper method to get the number of unique items in the baskets-dataset.
    """
    return set.union(*baskets)

def get_number_of_baskets(baskets):
    """
    Helper method to get the number of baskets. But you can also use len(baskets) 😂 I walk over the å to get the vatten.
    """
    return len(baskets)

We proceed with loading all baskets into a list of sets, where each set is a basket.

In [2]:
# NOTE: this could differ depending on what OS you are using (I'm on a Mac M1 and I'm on Windows10)
path_to_data = '../data/baskets.dat'

baskets = import_baskets(path_to_data)

print(f'Number of unique items: {len(get_unique_items(baskets))}')
print(f'Number of baskets: {len(baskets)}')

Number of unique items: 870
Number of baskets: 100000


For the creation of `L`<sub>`1`</sub> (frequent singletons) we use a dictionary that maps each item to the baskets it is in:
    
```python
    {
    0: [1,2,3], # item 0 is in baskets 1, 2 and 3
    1: [2, 3],  # item 1 is in baskets 2 and 3
    # and so on
    }
```

In [3]:
# get item and basket dictionary
def item_and_basket_dictionary(baskets):
    """
    Returns a dictionary with each each item as a key, and a list of all basket it appears in as a value.
    """
    # set up keys as all unique items
    keys = get_unique_items(baskets)

    # values will be lists of basket's number where they occur
    values = [[] for _ in range(len(get_unique_items(baskets)))]

    items = dict(zip(keys, values))

    # populate the dictionary
    for i, basket in enumerate(baskets):
        for item in basket:
            items[item].append(i)
    return items

The above dictionary is then used to find the support (in absolute number) for each singleton item by counting the number of baskets it is in.

In [4]:
# 'singletons' is the support for all singletons
singletons = {k: len(v) for k, v in item_and_basket_dictionary(baskets).items()}

Next is the filter step, and removing all items that do not meet the minimum support threshold:

$ \text{Minimum support} = 0.01 * \texttt{len(baskets)} $, in this case $0.01 * 100 000 = 1000$.

The result is `L`<sub>`1`</sub> - Frequent Singletons.

In [5]:
# set threshold as suggested in lecture
threshold = 0.01

# filter out all singletons with support below threshold
frequent_singletons = {k: v for k, v in singletons.items() if v / len(baskets) >= threshold}

# print the number of frequent singletons
print(f'Number of significant singletons: {len(frequent_singletons)} with a threshold of {threshold} (minimum support of {int(threshold * len(baskets))})')

Number of significant singletons: 375 with a threshold of 0.01 (minimum support of 1000)


For the second pass of the Apriori algorithm, we need a method to generate candidate itemsets. To this end, the course literature suggests the following:

> **From course literature (p.226):** During the second pass, we count all the pairs that *consists of two frequent items*. We call this table *frequent-items table*.

We then check if the pair is frequent, and if it is, we add it to the set of frequent itemsets.
The algorithm for generating candidate itemsets is as follows:

In [6]:
import numpy as np

def get_frequent_items_table(baskets, frequent_items):
    """
    Returns a table of frequent items, where each row is a frequent item and each column is a basket.
    frequent_items is a list where all frequent singletons has a uniqe m-value.
    """
    # NOTE there are 870 uniqe items assigned to integers [0, 1000]. 
    # This step is usually done by as a first step in the apriori alogoritm.
    frequent_items_table = np.zeros((max(map(max, baskets))) + 1, dtype=int)
    # assign 'm' as a unique values, used to mark the frequent items
    m = 0
    # identify all frequent items
    for item in frequent_items:
        # NOTE why dont we just use boolean values, is there a reason to use integers?
        frequent_items_table[item] = m
        m += 1
    return frequent_items_table

The Apriori algorithm requires some more methods; we have the candidate $k$-tuples are frequent.

We still follow the outline as suggested in the course literature (p.226):

> 1. For each basket, look in the frequent-items table to see which of its items are frequent.
> 2. In a double loop, generate all pairs of frequent items in that basket.
> 3. For each such pair, add one to its count in the data structure used to store counts.

So we are going to look through each basket, and, by using the previous pass frequent items,filter a subset of the items to produce all combinations. The we store those in a dictionary, and increment the count for each combination. This approach is a bit different from the one found in the 1994 paper, as they propose generating all combinations from items in the previous pass. Our approach it is more efficient.

To generate all unordered pairs of frequent items in a basket, we use the `combinations` function from the `itertools` library. We use `frozensets` since they are a hashable type, and can be used as keys in a dictionary.

In [7]:
from itertools import combinations

def filter_frequent_items_in_basket(basket, frequent_items_table):
    """
    [HELPER FUNCTION] Return the frequent items in a basket.
    """
    return [item for item in basket if frequent_items_table[item] != 0]

def generate_all_combinations(items, k=2):
    """
    [HELPER FUNCTION] Generate all pairs of frequent items in a basket.
    """
    return [frozenset(pair) for pair in combinations(items, k)]

def find_candidate_tuples(baskets, frequent_items_table, k=2):
    """
    Find all candidate k-tuples of frequent items.
    """
    candidate_pairs = {}
    for basket in baskets:
        # filter out all frequent items in the basket
        frequent_items_in_basket = filter_frequent_items_in_basket(basket, frequent_items_table)
        # genereate all combinations of frequent items in the basket
        basket_pairs = generate_all_combinations(frequent_items_in_basket, k=k)
        for pair in basket_pairs:
            if pair not in candidate_pairs:
                candidate_pairs[pair] = 1
            else:
                candidate_pairs[pair] += 1
    return candidate_pairs


def filter_candidate_tuples(baskets, candidates, threshold):
    """
    Filter out all candidate k-tuples that do not meet the threshold.
    """
    #return dict(filter(lambda x: x[1] >= threshold, candidate_pairs.items())) # TODO är filter bättre?
    return {pair: freq for pair, freq in candidates.items() if freq >= threshold*len(baskets)}

## The Apriori algorithm
The `apriori()`-function mimic the pseudocode from the 1994 article. In order to speed up the algorithm, we use the suggested approach in the course literature, and use the frequent-items table to filter out infrequent items in all paskets. We then produce all $k$-tuples using the remaining (frequent) items. This assumption holds due to the *monotonicity of itemsets*.

### Monotonicity of Itemsets

According to the paper and ch 6 of the course literature, the effectiveness of the Apriori algorithm depends on the monotonicity of the itemsets. 

> **Definition**: If a set $I$ of items is frequent, then so is the powerset of $I$ are also frequent (remember, the **powerset of $I$ is the set of all subsets of $I$**)

For example, if the itemset $I = \{1, 2, 3\}$ is frequent, then so are *all* subsets, $\mathscr{P}(I) = \{\{1\}, \{2\}, \{3\}, \{1, 2\}, \{1, 3\}, \{3, 2\}\}$, of $I$.

In [9]:
def apriori(baskets, singletons, threshold=0.01):
    """
    Find frequent itemsets using the Apriori algorithm.
    L1 ("Large 1-tons") is a frozenset of frequent singletons and their frequencies.
    """
    # find frequent singletons
    frequent_singletons = {k: v for k, v in singletons.items() if v / len(baskets) >= threshold}
    # convert to a frozenset
    L1 = frozenset(frequent_singletons.keys())
    # set up initial values
    k, L = 2, L1
    # this while-loop is just like the pipeline in the lecture
    while True:
        # genereate frequent-items table
        fit = get_frequent_items_table(baskets, L)
        # find all candidate pairs
        candidates = find_candidate_tuples(baskets, fit, k=k)
        # filter candidate pairs
        filtered_candidates = filter_candidate_tuples(baskets=baskets, candidates=candidates, threshold=threshold)
        # if no filter candidate pairs, break loop
        if len(filtered_candidates) == 0:
            break
        # else, set L to filtered candidates
        print(f'Number of {k}-tuples: {len(filtered_candidates)}')
        print(f'Number of unique items in {k}-tuples: {len(frozenset.union(*filtered_candidates))}\n')
        L = frozenset.union(*filtered_candidates.keys())
        k += 1

The method does what the pipeline suggest, and break the problem into smaller, handlable chunks.

![Apriori pipeline](../imgs/apriori_pipeline.jpeg)

In [10]:
apriori(baskets, singletons, threshold=threshold)

Number of 2-tuples: 9
Number of unique items in 2-tuples: 12



Now we collect all of the above methods in one neat class that stores all results in a dictionary called `frequent_itemsets`
```python
# one line to rule them all
frequent_itemsets = Apriori(baskets, min_support=0.01).frequent_itemsets
```

In [11]:
import numpy as np
from itertools import combinations

class Apriori:
    def __init__(self, baskets, threshold=0.01):
        self.threshold = threshold
        self.max_item_id = 0
        singletons = {k: len(v) for k, v in self.item_and_basket_dictionary(baskets).items()}
        self.frequent_itemsets = self.apriori(baskets=baskets, singletons=singletons, threshold=self.threshold)

    def get_unique_items(self, baskets):
        """
        [HELPER METHOD] Get the number of unique items in the baskets-dataset.
        """
        return set.union(*baskets)
        
    def item_and_basket_dictionary(self, baskets):
        """
        Returns a dictionary with each each item as a key, 
        and a list of all baskets it appears in as a value.
        """
        # set up keys as all unique items
        keys = self.get_unique_items(baskets)
        self.max_item_id = max(keys)

        # values will be lists of basket's number where they occur
        values = [[] for _ in range(len(get_unique_items(baskets)))]

        items = dict(zip(keys, values))

        # populate the dictionary
        for i, basket in enumerate(baskets):
            for item in basket:
                items[item].append(i)
        return items

    def get_frequent_items_table(self, frequent_items):
        """
        Returns a table of frequent items, where each row is a frequent item and each column is a basket.
        frequent_items is a list where all frequent singletons has a uniqe m-value.
        """
        # NOTE there are 870 uniqe items assigned to integers [0, 1000]. 
        # This step is usually done by as a first step in the apriori alogoritm.
        frequent_items_table = np.zeros(self.max_item_id + 1, dtype=int) 
        # assign 'm' as a unique values, used to mark the frequent items
        m = 1
        # identify all frequent items
        for item in frequent_items:
            # NOTE why dont we just use boolean values, is there a reason to use integers?
            frequent_items_table[item] = m
            m += 1
        return frequent_items_table

    def filter_frequent_items_in_basket(self, basket, frequent_items_table):
        """
        [HELPER FUNCTION] Return the frequent items in a basket.
        """
        return [item for item in basket if frequent_items_table[item] != 0]

    def generate_all_combinations(self, items, k=2):
        """
        [HELPER FUNCTION] Generate all pairs of frequent items in a basket.
        """
        return [frozenset(pair) for pair in combinations(items, k)]

    def find_candidate_tuples(self, baskets, frequent_items_table, k=2):
        """
        Find all candidate k-tuples of frequent items.
        """
        candidate_pairs = {}
        for basket in baskets:
            # filter out all frequent items in the basket
            frequent_items_in_basket = self.filter_frequent_items_in_basket(basket, frequent_items_table)
            # genereate all combinations of frequent items in the basket
            basket_pairs = self.generate_all_combinations(frequent_items_in_basket, k=k)
            for pair in basket_pairs:
                if pair not in candidate_pairs:
                    candidate_pairs[pair] = 1
                else:
                    candidate_pairs[pair] += 1
        return candidate_pairs


    def filter_candidate_tuples(self, baskets, candidates, threshold):
        """
        Filter out all candidate k-tuples that do not meet the threshold.
        """
        #return dict(filter(lambda x: x[1] >= threshold, candidate_pairs.items())) # TODO är filter bättre?
        return {pair: freq for pair, freq in candidates.items() if freq > threshold*len(baskets)}

    def printing_the_print(self, k, filtered_candidates):
        print(f'Number of {k}-tuples: {len(filtered_candidates)}')
        union = len(frozenset.union(*filtered_candidates))
        print(f'Number of unique items in {k}-tuples: {union}\n')

    def apriori(self, baskets, singletons, threshold=0.01):
        """
        Find frequent itemsets using the Apriori algorithm.
        L1 ("Large 1-tons") is a frozenset of frequent singletons and their frequencies.
        """
        # special case - find frequent singletons
        frequent_singletons = {frozenset([k]): v for k, v in singletons.items() if v / len(baskets) > threshold}
        # convert to a frozenset
        L1 = frozenset.union(*frequent_singletons.keys())
        # print progress
        self.printing_the_print(1, frequent_singletons)
        results = {1: frequent_singletons}
        # set up initial values
        k, L = 2, L1
        # this while-loop is just like the pipeline in the lecture
        while True:
            # genereate frequent-items table
            fit = self.get_frequent_items_table(L)
            # find all candidate pairs
            candidates = self.find_candidate_tuples(baskets, fit, k=k)
            # filter candidate pairs
            filtered_candidates = self.filter_candidate_tuples(baskets=baskets, candidates=candidates, threshold=threshold)
            # if no filter candidate pairs, break loop
            if len(filtered_candidates) == 0:
                break
            # print the progress
            self.printing_the_print(k, filtered_candidates)
            # else, set L to filtered candidates and continue
            L = frozenset.union(*filtered_candidates.keys())
            results[k] = filtered_candidates
            k += 1
        return results

    

Here we go:

In [12]:
threshold = 0.005
frequent_itemsets = Apriori(baskets, threshold=threshold).frequent_itemsets

Number of 1-tuples: 569
Number of unique items in 1-tuples: 569

Number of 2-tuples: 339
Number of unique items in 2-tuples: 210

Number of 3-tuples: 109
Number of unique items in 3-tuples: 92

Number of 4-tuples: 42
Number of unique items in 4-tuples: 55

Number of 5-tuples: 9
Number of unique items in 5-tuples: 20



# Finding Association Rules

Now we find useful assosciation rules. We use the `AssociationRules` class to do this. The class takes the frequent itemsets as input, and finds all association rules that meet the minimum confidence threshold.

In [12]:
class AssosciationRules: 
    def __init__(self, frequent_itemsets):
        self.frequent_itemsets = frequent_itemsets
        self.rules = self.generate_rules()
        print(f'Number of rules: {len(self.rules)}')
        self.interesting_rules = None
        
    def generate_rules(self):
        rules = {}
        # k is the length of the frequent itemset
        # v is the frequent itemset
        for k, v in self.frequent_itemsets.items():
            # skip singletons
            if k == 1:
                continue
            for itemset, freq in v.items():
                for item in itemset:
                    # left side of the rule
                    antecedent = frozenset(itemset - frozenset([item]))
                    # right side of the rule
                    consequent = frozenset([item])
                    # numerator - freq: support(I union j)
                    # denominator: support(I)
                    confidence = freq / (self.frequent_itemsets[len(antecedent)][antecedent])
                    # store them in a dictionary with the rule as key and confidence as value
                    rules[(antecedent, consequent)] = confidence
        return rules
    
    def print_rules(self, n=10, confidence_threshold=0.5):
        for i, (rule, confidence) in enumerate(sorted(self.rules.items(), key=lambda x: x[1], reverse=True)):
            if i == n:
                break
            if confidence < confidence_threshold:
                continue
            else:
                antecedent, consequent = rule
                print(f'{antecedent} -> {consequent} (confidence: {confidence:.3f})')

    def generate_interesting_rules(self, baskets, interest_threshold=0.5):
        interesting_rules = {}
        num_baskets = len(baskets)
        for rule in self.rules:
            antecedent, consequent = rule
            confidence = self.rules[rule]
            
            support = self.frequent_itemsets[int(len(consequent))][consequent]
            prob_consequent = support/num_baskets
            interest = confidence - prob_consequent

            if interest > interest_threshold or interest < -interest_threshold:
                interesting_rules[(antecedent, consequent)] = interest

        print(f'\nNumber of interesting rules: {len(interesting_rules)}')
        self.interesting_rules = interesting_rules

    def print_interesting_rules(self, n=10):
        for i, (rule, interest) in enumerate(sorted(self.interesting_rules.items(), key=lambda x: x[1], reverse=True)):
            if i == n:
                break
            else:
                antecedent, consequent = rule
                print(f'{antecedent} -> {consequent} (interest: {interest:.3f})')
       
    def get_rules(self):
        return self.rules
    
    def get_frequent_itemsets(self):
        return self.frequent_itemsets

To show our implementation, we run the follow cell.

In [13]:
threshold = 0.005
apriori = Apriori(baskets, threshold=threshold)
assosciation_rules = AssosciationRules(apriori.frequent_itemsets)
assosciation_rules.print_rules(n=5, confidence_threshold=0.5)
assosciation_rules.generate_interesting_rules(baskets, interest_threshold=0.5)
assosciation_rules.print_interesting_rules(n=5)

Number of 1-tuples: 569
Number of unique items in 1-tuples: 569

Number of 2-tuples: 339
Number of unique items in 2-tuples: 210

Number of 3-tuples: 109
Number of unique items in 3-tuples: 92

Number of 4-tuples: 42
Number of unique items in 4-tuples: 55

Number of 5-tuples: 9
Number of unique items in 5-tuples: 20

Number of rules: 1218
frozenset({960, 185, 678}) -> frozenset({471}) (confidence: 0.994)
frozenset({217, 546, 947}) -> frozenset({661}) (confidence: 0.991)
frozenset({217, 546, 923}) -> frozenset({661}) (confidence: 0.991)
frozenset({217, 546, 947, 923}) -> frozenset({661}) (confidence: 0.991)
frozenset({546, 923, 947}) -> frozenset({661}) (confidence: 0.990)

Number of interesting rules: 595
frozenset({960, 185, 471}) -> frozenset({678}) (interest: 0.975)
frozenset({217, 947, 923, 661}) -> frozenset({546}) (interest: 0.974)
frozenset({217, 947, 661}) -> frozenset({546}) (interest: 0.971)
frozenset({960, 678, 471}) -> frozenset({185}) (interest: 0.970)
frozenset({923, 947,

## Final note

There are further optimization to be done, such as using a *bitmap* to store the baskets, and using a *hash tree* to store the frequent itemsets. However, we don't believe those are the primary focus of this lab.